In [1]:
import sys
import os

notebook_path = os.getcwd() 
project_root = os.path.dirname(notebook_path)

if project_root not in sys.path:
    sys.path.insert(0, project_root)

In [2]:
import torch
import torch.nn as nn
from torch.utils.data import DataLoader, TensorDataset
import matplotlib.pyplot as plt
from copy import deepcopy

In [3]:
from models.filter_trajectory_ensemble import trajectory_StateBayesianKalmanNet
from systems.DynamicSystem import DynamicSystem
from training.utils import calculate_anees_vectorized, generate_data
import training.trainer as trainer

In [4]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Používané zařízení: {device}")

Používané zařízení: cuda


In [5]:
state_dim_2d = 2
obs_dim_2d = 2

F_base_2d = torch.tensor([[1.0, 1.0], 
                          [0.0, 1.0]])

svd_F = torch.linalg.svd(F_base_2d)
F_true_2d = F_base_2d / svd_F.S[0]

H_true_2d = torch.eye(obs_dim_2d)

Q_true_2d = torch.eye(state_dim_2d) * 0.5 # Šum procesu
R_true_2d = torch.eye(obs_dim_2d) * 0.1 # Šum měření

# Počáteční podmínky
Ex0_true_2d = torch.tensor([[1.0], [0.0]])
P0_true_2d = torch.eye(state_dim_2d) * 1.5
F_model_2d = F_true_2d
H_model_2d = H_true_2d
Q_model_2d = torch.eye(state_dim_2d) * 0.1
R_model_2d = R_true_2d
Ex0_model_2d = torch.tensor([[0.5], [0.5]])
P0_model_2d = torch.eye(state_dim_2d) * 1.0

print("\nInicializuji 2D Linear_Canonical systém (replikace autorů)...")
sys_true = DynamicSystem(
    state_dim=state_dim_2d, obs_dim=obs_dim_2d,
    Ex0=Ex0_true_2d, P0=P0_true_2d,
    Q=Q_true_2d, R=R_true_2d,
    F=F_true_2d, H=H_true_2d,
    device=device
)
sys_model = DynamicSystem(
    state_dim=state_dim_2d, obs_dim=obs_dim_2d,
    Ex0=Ex0_model_2d, P0=P0_model_2d,
    Q=Q_model_2d, R=R_model_2d,
    F=F_model_2d, H=H_model_2d,
    device=device
)
print("... 2D systém inicializován.")


Inicializuji 2D Linear_Canonical systém (replikace autorů)...
... 2D systém inicializován.


In [6]:
TRAIN_SEQ_LEN = 10      # Krátké sekvence pro stabilní trénink (TBPTT)
VALID_SEQ_LEN = 20      # Stejná délka pro konzistentní validaci
TEST_SEQ_LEN = 100      # Dlouhé sekvence pro testování generalizace

NUM_TRAIN_TRAJ = 500   # Hodně trénovacích příkladů
NUM_VALID_TRAJ = 200    # Dostatek pro spolehlivou validaci
NUM_TEST_TRAJ = 100     # Pro robustní vyhodnocení

BATCH_SIZE = 8         # Dobrý kompromis

x_train, y_train = generate_data(sys_true, num_trajectories=NUM_TRAIN_TRAJ, seq_len=TRAIN_SEQ_LEN)
x_val, y_val = generate_data(sys_true, num_trajectories=NUM_VALID_TRAJ, seq_len=VALID_SEQ_LEN)
x_test, y_test = generate_data(sys_true, num_trajectories=1, seq_len=TEST_SEQ_LEN)

train_dataset = TensorDataset(x_train, y_train)
val_dataset = TensorDataset(x_val, y_val)

train_loader = DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=BATCH_SIZE, shuffle=False)

In [7]:
import torch
import torch.nn as nn
from torch.utils.data import TensorDataset, DataLoader
import numpy as np
import os
import random
import csv
from datetime import datetime
import pandas as pd
from copy import deepcopy

model_config = {
    "hidden_size_multiplier": 10,
    "output_layer_multiplier": 4,
    "num_gru_layers": 1,
    "init_min_dropout": 0.8,
    "init_max_dropout": 0.9
}

train_config = {
    "total_train_iter": 800,
    "learning_rate": 1e-4,
    "clip_grad": 10.0,
    "J_samples": 20,
    "validation_period": 20,
    "logging_period": 20,
    "warmup_iterations":400 # Trénuj prvních 400 iterací jen na MSE
}

# =================================================================================
# KROK 3: SPUŠTĚNÍ JEDNOHO TRÉNINKOVÉHO BĚHU
# =================================================================================

print("="*80)
print("Spouštím jeden plnohodnotný tréninkový běh...")
print(f"Parametry modelu: {model_config}")
print(f"Parametry tréninku: {train_config}")
print("="*80)

# Nastavení seedu pro reprodukovatelnost tohoto běhu
torch.manual_seed(42)
np.random.seed(42)
random.seed(42)

# Vytvoření modelu
state_bkn_knet = trajectory_StateBayesianKalmanNet(
    sys_model,
    device=device,
    **model_config
).to(device)

# Spuštění tréninku
# Používáme `run_training_session`, která vrací slovník s výsledky
results = trainer.training_session_trajectory_with_gaussian_nll_training_fcn(model=state_bkn_knet,
    train_loader=train_loader,
    val_loader=val_loader,
    device=device,
    **train_config
)

# `run_training_session` automaticky načte nejlepší model zpět,
# takže `state_bkn_knet` nyní obsahuje váhy nejlepšího modelu.
trained_model = results['final_model']

print("\n" + "="*80)
print("TRÉNINK DOKONČEN - FINÁLNÍ VÝSLEDKY Z NEJLEPŠÍHO MODELU")
print("="*80)
print(f"Nejlepší model byl nalezen v iteraci: {results['best_iter']}")
# --- Změněné klíče, aby odpovídaly return statementu ---
print(f"Nejlepší dosažený validační ANEES: {results['best_val_anees']:.4f}")
print("--- Metriky odpovídající tomuto nejlepšímu modelu ---")
print(f"  MSE na validační sadě:       {results['best_val_mse']:.4f}")
print(f"  NLL na validační sadě:       {results['best_val_nll']:.4f}")
print("="*80)

# Nyní můžeš s `trained_model` pokračovat, například ho vyhodnotit na testovací sadě.

Spouštím jeden plnohodnotný tréninkový běh...
Parametry modelu: {'hidden_size_multiplier': 10, 'output_layer_multiplier': 4, 'num_gru_layers': 1, 'init_min_dropout': 0.8, 'init_max_dropout': 0.9}
Parametry tréninku: {'total_train_iter': 800, 'learning_rate': 0.0001, 'clip_grad': 10.0, 'J_samples': 20, 'validation_period': 20, 'logging_period': 20, 'warmup_iterations': 400}
--- Iteration [20/800] ---
    - Total Loss: 0.5354
    - NLL: 0.0000
    - Reg: 0.0068
    - p1=0.836, p2=0.853

--- Validace v iteraci 20 ---
  Průměrný MSE: 0.4639, Průměrný ANEES: 7.7876
  >>> Nové nejlepší VALIDAČNÍ ANEES! Ukládám model. <<<
--------------------------------------------------
--- Iteration [40/800] ---
    - Total Loss: 0.3353
    - NLL: 0.0000
    - Reg: 0.0067
    - p1=0.836, p2=0.853

--- Validace v iteraci 40 ---
  Průměrný MSE: 0.3494, Průměrný ANEES: 6.8860
  >>> Nové nejlepší VALIDAČNÍ ANEES! Ukládám model. <<<
--------------------------------------------------
--- Iteration [60/800] ---
 

KeyboardInterrupt: 

In [ ]:
import torch
import torch.nn.functional as F
import numpy as np
from torch.utils.data import TensorDataset, DataLoader

TEST_SEQ_LEN = 100
NUM_TEST_TRAJ = 20
J_SAMPLES_TEST = 25

# ==============================================================================
# 3. PŘÍPRAVA DAT
# ==============================================================================
print(f"\nGeneruji {NUM_TEST_TRAJ} testovacích trajektorií o délce {TEST_SEQ_LEN}...")
x_test, y_test = generate_data(sys_true, num_trajectories=NUM_TEST_TRAJ, seq_len=TEST_SEQ_LEN)
test_dataset = TensorDataset(x_test, y_test)
test_loader = DataLoader(test_dataset, batch_size=1, shuffle=False)
print("Generování dat dokončeno.")

# ==============================================================================
# 4. INICIALIZACE EKF
# ==============================================================================
ekf = ExtendedKalmanFilter(sys_model)
print("Extended Kalman Filter inicializován.")

# ==============================================================================
# 5. VYHODNOCOVACÍ SMYČKA
# ==============================================================================
all_x_true_cpu, all_x_hat_knet_cpu, all_P_hat_knet_cpu = [], [], []
all_x_hat_ekf_cpu, all_P_hat_ekf_cpu = [], []

print(f"\nVyhodnocuji modely na {NUM_TEST_TRAJ} testovacích trajektoriích...")

trained_model.train() 
with torch.no_grad():
    for i, (x_true_seq_batch, y_test_seq_batch) in enumerate(test_loader):
        y_test_seq_gpu = y_test_seq_batch.squeeze(0).to(device)
        x_true_seq_gpu = x_true_seq_batch.squeeze(0).to(device)
        
        # --- A. Vyhodnocení Bayesian KalmanNet ---
        initial_state = x_true_seq_gpu[0, :].unsqueeze(0)
        trained_model.reset(batch_size=1, initial_state=initial_state)

        x_hat_list_knet = []
        P_hat_list_knet = []

        for t in range(1, TEST_SEQ_LEN):
            y_t = y_test_seq_gpu[t, :].unsqueeze(0)
            x_filtered_t, _, _, x_ensemble_t = trained_model.step(y_t, J_SAMPLES_TEST)
            
            # `x_filtered_t` má tvar [1, state_dim], ponecháme ho tak
            x_hat_list_knet.append(x_filtered_t)
            
            x_ensemble_t_squeezed = x_ensemble_t.squeeze(0)
            x_mean_t = x_filtered_t.squeeze(0)
            diff = x_ensemble_t_squeezed - x_mean_t
            outer_products = diff.unsqueeze(-1) @ diff.unsqueeze(-2)
            P_hat_full_t = outer_products.mean(dim=0)
            P_hat_list_knet.append(P_hat_full_t)

        # ----- ZDE JE OPRAVA -----
        predictions_knet = torch.cat(x_hat_list_knet, dim=0) # Použijeme cat místo stack
        covariances_knet = torch.stack(P_hat_list_knet, dim=0) # Zde stack zůstává, je správně
        
        full_x_hat_knet = torch.cat([initial_state, predictions_knet], dim=0)
        P0_val = trained_model.system_model.P0.to(device)
        full_P_hat_knet = torch.cat([P0_val.unsqueeze(0), covariances_knet], dim=0)
        
        # --- B. Vyhodnocení Extended Kalman Filter ---
        Ex0_ekf = sys_model.Ex0.to(device)
        P0_ekf = sys_model.P0.to(device)
        ekf_results = ekf.process_sequence(y_test_seq_gpu, Ex0=Ex0_ekf, P0=P0_ekf)
        
        # Upraveno pro robustnost - procesujeme celou sekvenci, EKF si vezme první y jako první měření
        full_x_hat_ekf = torch.cat([Ex0_ekf.reshape(1, -1), ekf_results['x_filtered']], dim=0)
        full_P_hat_ekf = torch.cat([P0_ekf.unsqueeze(0), ekf_results['P_filtered']], dim=0)
        
        # --- C. Uložení výsledků na CPU ---
        # Srovnáme délky pro případ, že by se lišily o 1
        min_len = min(full_x_hat_knet.shape[0], full_x_hat_ekf.shape[0], x_true_seq_gpu.shape[0])
        
        all_x_true_cpu.append(x_true_seq_gpu[:min_len].cpu())
        all_x_hat_knet_cpu.append(full_x_hat_knet[:min_len].cpu())
        all_P_hat_knet_cpu.append(full_P_hat_knet[:min_len].cpu())
        all_x_hat_ekf_cpu.append(full_x_hat_ekf[:min_len].cpu())
        all_P_hat_ekf_cpu.append(full_P_hat_ekf[:min_len].cpu())

        print(f"Dokončena trajektorie {i + 1}/{NUM_TEST_TRAJ}...")

# ==============================================================================
# 6. FINÁLNÍ VÝPOČET A VÝPIS METRIK
# ==============================================================================
mse_knet_list, anees_knet_list = [], []
mse_ekf_list, anees_ekf_list = [], []

print("\nPočítám finální metriky...")

with torch.no_grad():
    for i in range(NUM_TEST_TRAJ):
        x_true = all_x_true_cpu[i]
        
        x_hat_knet = all_x_hat_knet_cpu[i]
        P_hat_knet = all_P_hat_knet_cpu[i]
        mse_knet_run = F.mse_loss(x_hat_knet[1:], x_true[1:]).item()
        anees_knet_run = calculate_anees_vectorized(x_true.unsqueeze(0), x_hat_knet.unsqueeze(0), P_hat_knet.unsqueeze(0))
        mse_knet_list.append(mse_knet_run)
        anees_knet_list.append(anees_knet_run)
        
        x_hat_ekf = all_x_hat_ekf_cpu[i]
        P_hat_ekf = all_P_hat_ekf_cpu[i]
        mse_ekf_run = F.mse_loss(x_hat_ekf[1:], x_true[1:]).item()
        anees_ekf_run = calculate_anees_vectorized(x_true.unsqueeze(0), x_hat_ekf.unsqueeze(0), P_hat_ekf.unsqueeze(0))
        mse_ekf_list.append(mse_ekf_run)
        anees_ekf_list.append(anees_ekf_run)

        print(f"Trajektorie {i+1:2d} | BKN MSE: {mse_knet_run:7.4f} | EKF MSE: {mse_ekf_run:7.4f} || BKN ANEES: {anees_knet_run:7.4f} | EKF ANEES: {anees_ekf_run:7.4f}")

avg_mse_knet = np.mean(mse_knet_list)
avg_anees_knet = np.mean(anees_knet_list)
avg_mse_ekf = np.mean(mse_ekf_list)
avg_anees_ekf = np.mean(anees_ekf_list)
state_dim_for_nees = all_x_true_cpu[0].shape[1]

print("\n" + "="*60)
print(f"FINÁLNÍ VÝSLEDKY (průměr přes {NUM_TEST_TRAJ} běhů)")
print("="*60)
print("\n--- Průměrná MSE (přesnost) ---")
print(f"Bayesian KNet:      {avg_mse_knet:.4f}")
print(f"Extended KF:        {avg_mse_ekf:.4f}")
print("\n--- Průměrný ANEES (kredibilita/kalibrace) ---")
print(f"Očekávaná hodnota:   {state_dim_for_nees:.4f}")
print("---------------------------------------------")
print(f"Bayesian KNet:      {avg_anees_knet:.4f}")
print(f"Extended KF:        {avg_anees_ekf:.4f}")
print("="*60)


Generuji 20 testovacích trajektorií o délce 100...
Generování dat dokončeno.
Extended Kalman Filter inicializován.

Vyhodnocuji modely na 20 testovacích trajektoriích...
Dokončena trajektorie 1/20...
Dokončena trajektorie 2/20...
Dokončena trajektorie 3/20...
Dokončena trajektorie 4/20...
Dokončena trajektorie 5/20...
Dokončena trajektorie 6/20...
Dokončena trajektorie 7/20...
Dokončena trajektorie 8/20...
Dokončena trajektorie 9/20...
Dokončena trajektorie 10/20...
Dokončena trajektorie 11/20...
Dokončena trajektorie 12/20...
Dokončena trajektorie 13/20...
Dokončena trajektorie 14/20...
Dokončena trajektorie 15/20...
Dokončena trajektorie 16/20...
Dokončena trajektorie 17/20...
Dokončena trajektorie 18/20...
Dokončena trajektorie 19/20...
Dokončena trajektorie 20/20...

Počítám finální metriky...
Trajektorie  1 | BKN MSE:  0.1167 | EKF MSE:  0.8162 || BKN ANEES:  2.1481 | EKF ANEES: 28.6130
Trajektorie  2 | BKN MSE:  0.1284 | EKF MSE:  0.6996 || BKN ANEES:  1.8526 | EKF ANEES: 24.394

In [ ]:
import torch
from torch.func import jacrev

class ExtendedKalmanFilter:
    def __init__(self, system_model):

        self.device = system_model.Q.device

        self.f = system_model.f
        self.h = system_model.h

        # Pokud je systém lineární, Jakobiány jsou konstantní matice F a H
        self.is_linear_f = getattr(system_model, 'is_linear_f', False)
        self.is_linear_h = getattr(system_model, 'is_linear_h', False)
        if self.is_linear_f:
            self.F = system_model.F
        if self.is_linear_h:
            self.H = system_model.H


        self.Q = system_model.Q.clone().detach().to(self.device)
        self.R = system_model.R.clone().detach().to(self.device)

        self.state_dim = self.Q.shape[0]
        self.obs_dim = self.R.shape[0]

        self.x_filtered_prev = None
        self.P_filtered_prev = None

        self.reset(system_model.Ex0, system_model.P0)

    def reset(self, Ex0=None, P0=None):
        """
        Inicializuje nebo resetuje stav filtru.
        """
        if Ex0 is not None:
            # Ujistíme se, že má správný tvar [dim, 1]
            self.x_filtered_prev = Ex0.clone().detach().reshape(self.state_dim, 1)
        if P0 is not None:
            self.P_filtered_prev = P0.clone().detach()

    def predict_step(self, x_filtered, P_filtered):
        if self.is_linear_f:
            F_t = self.F
        else:
            F_t = jacrev(self.f)(x_filtered.squeeze()).reshape(self.state_dim, self.state_dim)
        
        # OPRAVA: Transponujeme vstup pro f() a výstup zpět.
        # [State_Dim, 1] -> .T -> [1, State_Dim] -> f() -> [1, State_Dim] -> .T -> [State_Dim, 1]
        x_predict = self.f(x_filtered.T).T

        P_predict = F_t @ P_filtered @ F_t.T + self.Q
        return x_predict, P_predict

    def update_step(self, x_predict, y_t, P_predict):
        y_t = y_t.reshape(self.obs_dim, 1)

        if self.is_linear_h:
            H_t = self.H
        else:
            H_t = jacrev(self.h)(x_predict.squeeze()).reshape(self.obs_dim, self.state_dim)

        # OPRAVA: Transponujeme vstup pro h() a výstup zpět.
        y_predict = self.h(x_predict.T).T
        innovation = y_t - y_predict
        
        S = H_t @ P_predict @ H_t.T + self.R
        K = P_predict @ H_t.T @ torch.linalg.inv(S)
        x_filtered = x_predict + K @ innovation

        I = torch.eye(self.state_dim, device=self.device)
        P_filtered = (I - K @ H_t) @ P_predict @ (I - K @ H_t).T + K @ self.R @ K.T

        return x_filtered, P_filtered, K, innovation

    def step(self, y_t):
        """
        Provede jeden kompletní krok filtrace (predict + update) pro online použití.
        """
        # 1. Predikce z uloženého interního stavu
        x_predict, P_predict = self.predict_step(self.x_filtered_prev, self.P_filtered_prev)

        # 2. Update s novým měřením
        x_filtered, P_filtered = self.update_step(x_predict, y_t, P_predict)

        # 3. Aktualizace interního stavu pro další volání
        self.x_filtered_prev = x_filtered
        self.P_filtered_prev = P_filtered

        return x_filtered, P_filtered

    def process_sequence(self, y_seq, Ex0=None, P0=None):
            """
            Zpracuje celou sekvenci měření `y_seq` (offline) a vrátí detailní historii.
            """
            # Pokud nejsou zadány, použije defaultní hodnoty z `__init__`
            x_est = Ex0.clone().detach().reshape(self.state_dim, 1) if Ex0 is not None else self.x_filtered_prev.clone()
            P_est = P0.clone().detach() if P0 is not None else self.P_filtered_prev.clone()

            seq_len = y_seq.shape[0]

            x_filtered_history = torch.zeros(seq_len, self.state_dim, device=self.device)
            P_filtered_history = torch.zeros(seq_len, self.state_dim, self.state_dim, device=self.device)
            x_predict_history = torch.zeros(seq_len, self.state_dim, device=self.device)
            P_predict_history = torch.zeros(seq_len, self.state_dim, self.state_dim, device=self.device)
            kalman_gain_history = torch.zeros(seq_len, self.state_dim, self.obs_dim, device=self.device)
            innovation_history = torch.zeros(seq_len, self.obs_dim, device=self.device)

            for t in range(seq_len):
                # 1. Predict
                x_predict, P_predict = self.predict_step(x_est, P_est)

                # 2. Update
                x_est, P_est, K, innovation = self.update_step(x_predict, y_seq[t], P_predict)

                x_filtered_history[t] = x_est.squeeze()
                P_filtered_history[t] = P_est
                x_predict_history[t] = x_predict.squeeze()
                P_predict_history[t] = P_predict
                kalman_gain_history[t] = K
                innovation_history[t] = innovation.squeeze()

            return {
                'x_filtered': x_filtered_history,
                'P_filtered': P_filtered_history,
                'x_predict': x_predict_history,
                'P_predict': P_predict_history,
                'Kalman_gain': kalman_gain_history,
                'innovation': innovation_history
            }